# Game Play Student Performance 1
### First notebook
### Get baseline Brier Score

In [1]:
import pandas as pd
import numpy as np

In [2]:
raw_data_train = pd.read_csv("./student_perf_data_raw/train.csv")
raw_data_test = pd.read_csv("./student_perf_data_raw/test.csv")

In [3]:
#combine train and test set for feature engineering use later
raw_data_train['label'] = 'train'
raw_data_test['label'] = 'test'
combined_set = pd.concat([raw_data_train, raw_data_test], axis=0)
combined_set.tail()

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,...,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group,label,session_level
3723,20090312331414616,1001,1581679,map_hover,basic,22,NaN,NaN,NaN,NaN,...,NaN,tunic.wildlife,tunic.historicalsociety.entry,NaN,0,0,1,13-22,test,8.0
3724,20090312331414616,1002,1583044,map_hover,basic,22,NaN,NaN,NaN,NaN,...,NaN,tunic.capitol_2,tunic.historicalsociety.entry,NaN,0,0,1,13-22,test,8.0
3725,20090312331414616,1003,1583410,map_click,undefined,22,NaN,483.726363,-3.880047,456.0,...,NaN,tunic.capitol_2,tunic.historicalsociety.entry,NaN,0,0,1,13-22,test,8.0
3726,20090312331414616,1004,1585841,navigate_click,undefined,22,NaN,192.372139,38.216178,383.0,...,NaN,chap4_finale_c,tunic.capitol_2.hall,NaN,0,0,1,13-22,test,8.0
3727,20090312331414616,1005,1587810,checkpoint,basic,22,NaN,NaN,NaN,NaN,...,NaN,chap4_finale_c,tunic.capitol_2.hall,NaN,0,0,1,13-22,test,8.0


# Get random set to speed up process

In [4]:
unique_sessions = combined_set.loc[:, ['session_id']].drop_duplicates()
random_set = unique_sessions.sample(frac=0.1, replace=False).reset_index(drop=True)

In [5]:
# Set combined set to random set to reduce size
combined_set = random_set.merge(combined_set.rename_axis('temp_index').reset_index(), how='left', on='session_id').set_index('temp_index', drop=True)
combined_set.index.names = [""]

In [6]:
combined_set.tail()

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,...,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group,label,session_level
,,,,,,,,,,,,,,,,,,,,,
7529132,21010509530808210,1515,2205332,map_hover,basic,22,NaN,NaN,NaN,NaN,...,NaN,tunic.capitol_2,tunic.historicalsociety.entry,NaN,0,0,0,13-22,train,NaN
7529133,21010509530808210,1516,2206546,map_click,undefined,22,NaN,464.183857,7.514327,444.0,...,NaN,tunic.capitol_2,tunic.historicalsociety.entry,NaN,0,0,0,13-22,train,NaN
7529134,21010509530808210,1517,2207628,navigate_click,undefined,22,NaN,283.281230,-42.086853,443.0,...,NaN,chap4_finale_c,tunic.capitol_2.hall,NaN,0,0,0,13-22,train,NaN
7529135,21010509530808210,1518,2209460,navigate_click,undefined,22,NaN,251.120357,42.856754,549.0,...,NaN,chap4_finale_c,tunic.capitol_2.hall,NaN,0,0,0,13-22,train,NaN
7529136,21010509530808210,1519,2209610,checkpoint,basic,22,NaN,NaN,NaN,NaN,...,NaN,chap4_finale_c,tunic.capitol_2.hall,NaN,0,0,0,13-22,train,NaN


In [7]:
len(combined_set)

2605503

# Exploratory Analysis

In [8]:
explore_set = combined_set[combined_set['label']=='train']

In [9]:
explore_set.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2605503 entries, 21702656 to 7529136
Data columns (total 22 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   session_id      2605503 non-null  int64  
 1   index           2605503 non-null  int64  
 2   elapsed_time    2605503 non-null  int64  
 3   event_name      2605503 non-null  object 
 4   name            2605503 non-null  object 
 5   level           2605503 non-null  int64  
 6   page            56819 non-null    float64
 7   room_coor_x     2400258 non-null  float64
 8   room_coor_y     2400258 non-null  float64
 9   screen_coor_x   2400258 non-null  float64
 10  screen_coor_y   2400258 non-null  float64
 11  hover_duration  198143 non-null   float64
 12  text            959067 non-null   object 
 13  fqid            1789891 non-null  object 
 14  room_fqid       2605503 non-null  object 
 15  text_fqid       959068 non-null   object 
 16  fullscreen      2605503 non-n

In [10]:
# Consider to split event name into event and action
# When both room coordinates are positive, room is in sight. x
### Might be useful to know how many steps to room in sight for each level / event
### Might also be useful to know how many steps stayed when room is in sight
# Good to know time used at each step (index) x
# Good to know how many of each even, like how many person talked to (clicked), how many maps seen etc. x
# Good to know how many times the same map / person is clicked. (repeated actions) x
# Good to know if someone uses music / fullscreen or change halfway (use avg) x
# Good to know if text is a question 
# Good to know page access count for each session / level / level grp
# Good to know lead lag event_name
# Good to know text info after extracting room/fqid info

### Prelim Feature Engineering prior to aggregation

In [11]:
# Split event_name into event and action
combined_set["event"] = combined_set["event_name"].apply(lambda x: x.split("_")[0])
combined_set["action"] = combined_set["event_name"].apply(lambda x: x.split("_")[-1])
combined_set.tail()

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,...,room_fqid,text_fqid,fullscreen,hq,music,level_group,label,session_level,event,action
,,,,,,,,,,,,,,,,,,,,,
7529132,21010509530808210,1515,2205332,map_hover,basic,22,NaN,NaN,NaN,NaN,...,tunic.historicalsociety.entry,NaN,0,0,0,13-22,train,NaN,map,hover
7529133,21010509530808210,1516,2206546,map_click,undefined,22,NaN,464.183857,7.514327,444.0,...,tunic.historicalsociety.entry,NaN,0,0,0,13-22,train,NaN,map,click
7529134,21010509530808210,1517,2207628,navigate_click,undefined,22,NaN,283.281230,-42.086853,443.0,...,tunic.capitol_2.hall,NaN,0,0,0,13-22,train,NaN,navigate,click
7529135,21010509530808210,1518,2209460,navigate_click,undefined,22,NaN,251.120357,42.856754,549.0,...,tunic.capitol_2.hall,NaN,0,0,0,13-22,train,NaN,navigate,click
7529136,21010509530808210,1519,2209610,checkpoint,basic,22,NaN,NaN,NaN,NaN,...,tunic.capitol_2.hall,NaN,0,0,0,13-22,train,NaN,checkpoint,checkpoint


In [12]:
# Check if room in sight, and general positional info
combined_set['room_in_sight'] = np.where((combined_set['room_coor_x']>0) & (combined_set['room_coor_y']>0), True, False)
combined_set['room_position_info'] =  abs(combined_set['screen_coor_x']-combined_set['room_coor_x'])+abs(combined_set['screen_coor_y']-combined_set['room_coor_y'])
combined_set.tail()

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,...,fullscreen,hq,music,level_group,label,session_level,event,action,room_in_sight,room_position_info
,,,,,,,,,,,,,,,,,,,,,
7529132,21010509530808210,1515,2205332,map_hover,basic,22,NaN,NaN,NaN,NaN,...,0,0,0,13-22,train,NaN,map,hover,False,NaN
7529133,21010509530808210,1516,2206546,map_click,undefined,22,NaN,464.183857,7.514327,444.0,...,0,0,0,13-22,train,NaN,map,click,True,337.669530
7529134,21010509530808210,1517,2207628,navigate_click,undefined,22,NaN,283.281230,-42.086853,443.0,...,0,0,0,13-22,train,NaN,navigate,click,False,526.805622
7529135,21010509530808210,1518,2209460,navigate_click,undefined,22,NaN,251.120357,42.856754,549.0,...,0,0,0,13-22,train,NaN,navigate,click,True,534.022889
7529136,21010509530808210,1519,2209610,checkpoint,basic,22,NaN,NaN,NaN,NaN,...,0,0,0,13-22,train,NaN,checkpoint,checkpoint,False,NaN


In [13]:
# Get steps for level and level group
combined_set['step_count'] = combined_set.groupby(['session_id']).cumcount()
combined_set['level_step_count'] = combined_set.groupby(['session_id', 'level']).cumcount()
combined_set['levelgrp_step_count'] = combined_set.groupby(['session_id', 'level_group']).cumcount()
combined_set.tail()

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,...,level_group,label,session_level,event,action,room_in_sight,room_position_info,step_count,level_step_count,levelgrp_step_count
,,,,,,,,,,,,,,,,,,,,,
7529132,21010509530808210,1515,2205332,map_hover,basic,22,NaN,NaN,NaN,NaN,...,13-22,train,NaN,map,hover,False,NaN,1462,29,680
7529133,21010509530808210,1516,2206546,map_click,undefined,22,NaN,464.183857,7.514327,444.0,...,13-22,train,NaN,map,click,True,337.669530,1463,30,681
7529134,21010509530808210,1517,2207628,navigate_click,undefined,22,NaN,283.281230,-42.086853,443.0,...,13-22,train,NaN,navigate,click,False,526.805622,1464,31,682
7529135,21010509530808210,1518,2209460,navigate_click,undefined,22,NaN,251.120357,42.856754,549.0,...,13-22,train,NaN,navigate,click,True,534.022889,1465,32,683
7529136,21010509530808210,1519,2209610,checkpoint,basic,22,NaN,NaN,NaN,NaN,...,13-22,train,NaN,checkpoint,checkpoint,False,NaN,1466,33,684


In [14]:
# Get time use at each step for level and level group
combined_set['step_time_used'] = combined_set.groupby(['session_id'])['elapsed_time'].diff().fillna(0)
combined_set['level_step_time_used'] = combined_set.groupby(['session_id', 'level'])['elapsed_time'].diff().fillna(0)
combined_set['levelgrp_step_time_used'] = combined_set.groupby(['session_id', 'level_group'])['elapsed_time'].diff().fillna(0)
combined_set['level_elapsed_time'] = combined_set.groupby(['session_id', 'level'])['level_step_time_used'].cumsum()
combined_set['levelgrp_elapsed_time'] = combined_set.groupby(['session_id', 'level_group'])['levelgrp_step_time_used'].cumsum()
combined_set.head()

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,...,room_in_sight,room_position_info,step_count,level_step_count,levelgrp_step_count,step_time_used,level_step_time_used,levelgrp_step_time_used,level_elapsed_time,levelgrp_elapsed_time
,,,,,,,,,,,,,,,,,,,,,
21702656,22010214074668410,0,0,cutscene_click,basic,0,NaN,-384.991405,98.685314,409.0,...,False,931.306091,0,0,0,0.0,0.0,0.0,0.0,0.0
21702657,22010214074668410,1,374,person_click,basic,0,NaN,-384.991405,98.685314,409.0,...,False,931.306091,1,1,1,374.0,374.0,374.0,374.0,374.0
21702658,22010214074668410,2,756,person_click,basic,0,NaN,-233.991405,81.685314,560.0,...,False,965.306091,2,2,2,382.0,382.0,382.0,756.0,756.0
21702659,22010214074668410,3,4097,person_click,basic,0,NaN,-181.991405,29.685314,612.0,...,False,1069.306091,3,3,3,3341.0,3341.0,3341.0,4097.0,4097.0
21702660,22010214074668410,4,6507,person_click,basic,0,NaN,-114.991405,-36.314686,679.0,...,False,1201.306091,4,4,4,2410.0,2410.0,2410.0,6507.0,6507.0


In [15]:
# Get event type count for session, level and level grp 
#event
combined_set['session_event_count'] = combined_set.groupby(['session_id', 'event'])['event'].transform('count')
combined_set['level_event_count'] = combined_set.groupby(['session_id', 'level', 'event'])['event'].transform('count')
combined_set['levelgrp_event_count'] = combined_set.groupby(['session_id', 'level_group', 'event'])['event'].transform('count')
#action
combined_set['session_action_count'] = combined_set.groupby(['session_id', 'action'])['action'].transform('count')
combined_set['level_action_count'] = combined_set.groupby(['session_id', 'level', 'action'])['action'].transform('count')
combined_set['levelgrp_action_count'] = combined_set.groupby(['session_id', 'level_group', 'action'])['action'].transform('count')

In [16]:
# Get count of repeated actions
#repeated actions
combined_set['session_rptevent_count'] = combined_set.groupby(['session_id', 'event_name', 'fqid'])['fqid'].transform('count')
combined_set['level_rptevent_count'] = combined_set.groupby(['session_id', 'level', 'event_name', 'fqid'])['fqid'].transform('count')
combined_set['levelgrp_rptevent_count'] = combined_set.groupby(['session_id', 'level_group', 'event_name', 'fqid'])['fqid'].transform('count')

In [17]:
# Check music and fullscreen used for full session
combined_set['full_sess_music'] = combined_set.groupby(['session_id'])['music'].transform('mean')
combined_set['full_sess_fullscreen'] = combined_set.groupby(['session_id'])['fullscreen'].transform('mean')
combined_set.tail()

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,...,level_event_count,levelgrp_event_count,session_action_count,level_action_count,levelgrp_action_count,session_rptevent_count,level_rptevent_count,levelgrp_rptevent_count,full_sess_music,full_sess_fullscreen
,,,,,,,,,,,,,,,,,,,,,
7529132,21010509530808210,1515,2205332,map_hover,basic,22,NaN,NaN,NaN,NaN,...,4,33,105,3,41,6.0,1.0,6.0,0.0,0.0
7529133,21010509530808210,1516,2206546,map_click,undefined,22,NaN,464.183857,7.514327,444.0,...,4,33,1359,30,643,2.0,1.0,2.0,0.0,0.0
7529134,21010509530808210,1517,2207628,navigate_click,undefined,22,NaN,283.281230,-42.086853,443.0,...,29,364,1359,30,643,2.0,2.0,2.0,0.0,0.0
7529135,21010509530808210,1518,2209460,navigate_click,undefined,22,NaN,251.120357,42.856754,549.0,...,29,364,1359,30,643,2.0,2.0,2.0,0.0,0.0
7529136,21010509530808210,1519,2209610,checkpoint,basic,22,NaN,NaN,NaN,NaN,...,1,1,3,1,1,1.0,1.0,1.0,0.0,0.0


In [18]:
# Check if text is a question
combined_set['text_is_question'] = np.where((~combined_set['text'].isnull()) & (combined_set['text'].str.contains('\?')), True, False)

In [19]:
# Get page access count
combined_set['session_page_access_count'] = combined_set.groupby(['session_id', 'page'])['page'].transform('count')
combined_set['level_page_access_count'] = combined_set.groupby(['session_id', 'level', 'page'])['page'].transform('count')
combined_set['levelgrp_page_access_count'] = combined_set.groupby(['session_id', 'level_group', 'page'])['page'].transform('count')

In [20]:
# Get lead / lag event by level
combined_set['lead_eventname'] = combined_set['event_name'].shift(-1)
combined_set['lag_eventname'] = combined_set['event_name'].shift(1)
combined_set.head()

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,...,level_rptevent_count,levelgrp_rptevent_count,full_sess_music,full_sess_fullscreen,text_is_question,session_page_access_count,level_page_access_count,levelgrp_page_access_count,lead_eventname,lag_eventname
,,,,,,,,,,,,,,,,,,,,,
21702656,22010214074668410,0,0,cutscene_click,basic,0,NaN,-384.991405,98.685314,409.0,...,1.0,1.0,1.0,0.0,False,NaN,NaN,NaN,person_click,None
21702657,22010214074668410,1,374,person_click,basic,0,NaN,-384.991405,98.685314,409.0,...,5.0,14.0,1.0,0.0,True,NaN,NaN,NaN,person_click,cutscene_click
21702658,22010214074668410,2,756,person_click,basic,0,NaN,-233.991405,81.685314,560.0,...,5.0,14.0,1.0,0.0,False,NaN,NaN,NaN,person_click,person_click
21702659,22010214074668410,3,4097,person_click,basic,0,NaN,-181.991405,29.685314,612.0,...,5.0,14.0,1.0,0.0,False,NaN,NaN,NaN,person_click,person_click
21702660,22010214074668410,4,6507,person_click,basic,0,NaN,-114.991405,-36.314686,679.0,...,5.0,14.0,1.0,0.0,False,NaN,NaN,NaN,person_click,person_click


In [21]:
# Get additional text info if available
fqid = combined_set['fqid'].str.lower()
text_fqid = combined_set['text_fqid'].str.lower()
room_fqid = combined_set['room_fqid'].str.lower()      

In [22]:
df_temp = pd.DataFrame({'fqid':fqid, 'text_fqid':text_fqid})
df_checked = df_temp.apply(lambda x: [word for word in str(x.text_fqid).split(".") if word not in str(x.fqid).split('.')] , axis=1) 

In [23]:
df_temp = pd.DataFrame({'room_fqid':room_fqid, 'df_checked':df_checked})
df_temp['df_checked'] = ['.'.join(map(str,l)) for l in df_temp['df_checked']]
df_temp

,room_fqid,df_checked
,,
21702656,tunic.historicalsociety.closet,tunic.historicalsociety.closet
21702657,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro_0_cs_0
21702658,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro_0_cs_0
21702659,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro_0_cs_0
21702660,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro_0_cs_0
...,...,...
7529132,tunic.historicalsociety.entry,nan
7529133,tunic.historicalsociety.entry,nan
7529134,tunic.capitol_2.hall,nan


In [24]:
df_checked = df_temp.apply(lambda x: [word for word in str(x.df_checked).split(".") if word not in str(x.room_fqid).split('.')] , axis=1) 
df_checked


21702656                []
21702657    [intro_0_cs_0]
21702658    [intro_0_cs_0]
21702659    [intro_0_cs_0]
21702660    [intro_0_cs_0]
                 ...      
7529132              [nan]
7529133              [nan]
7529134              [nan]
7529135              [nan]
7529136              [nan]
Length: 2605503, dtype: object

In [25]:
df_checked = ['.'.join(map(str,l)) for l in df_checked]

In [27]:
combined_set['addition_info_fr_id'] = df_checked
combined_set.head(50)

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,...,levelgrp_rptevent_count,full_sess_music,full_sess_fullscreen,text_is_question,session_page_access_count,level_page_access_count,levelgrp_page_access_count,lead_eventname,lag_eventname,addition_info_fr_id
,,,,,,,,,,,,,,,,,,,,,
21702656,22010214074668410,0,0,cutscene_click,basic,0,NaN,-384.991405,98.685314,409.0,...,1.0,1.0,0.0,False,NaN,NaN,NaN,person_click,None,
21702657,22010214074668410,1,374,person_click,basic,0,NaN,-384.991405,98.685314,409.0,...,14.0,1.0,0.0,True,NaN,NaN,NaN,person_click,cutscene_click,intro_0_cs_0
21702658,22010214074668410,2,756,person_click,basic,0,NaN,-233.991405,81.685314,560.0,...,14.0,1.0,0.0,False,NaN,NaN,NaN,person_click,person_click,intro_0_cs_0
21702659,22010214074668410,3,4097,person_click,basic,0,NaN,-181.991405,29.685314,612.0,...,14.0,1.0,0.0,False,NaN,NaN,NaN,person_click,person_click,intro_0_cs_0
21702660,22010214074668410,4,6507,person_click,basic,0,NaN,-114.991405,-36.314686,679.0,...,14.0,1.0,0.0,False,NaN,NaN,NaN,person_click,person_click,intro_0_cs_0
21702661,22010214074668410,5,10007,person_click,basic,0,NaN,-218.991405,-71.314686,575.0,...,14.0,1.0,0.0,False,NaN,NaN,NaN,person_click,person_click,intro_0_cs_0
21702662,22010214074668410,6,12985,person_click,basic,0,NaN,370.335798,-119.326175,480.0,...,4.0,1.0,0.0,False,NaN,NaN,NaN,person_click,person_click,intro_0_cs_0
21702663,22010214074668410,7,14973,person_click,basic,0,NaN,386.963685,-105.737468,491.0,...,4.0,1.0,0.0,False,NaN,NaN,NaN,person_click,person_click,intro_0_cs_0
21702664,22010214074668410,8,17182,person_click,basic,0,NaN,386.115143,-107.748537,490.0,...,4.0,1.0,0.0,True,NaN,NaN,NaN,person_click,person_click,intro_0_cs_0


In [31]:
# combined_set.to_csv('./student_perf1/after_interim_feature_engineering.csv')

### Session Aggregation (By Session, level_grp and LABEL)

In [29]:
combined_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2605503 entries, 21702656 to 7529136
Data columns (total 52 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   session_id                  int64  
 1   index                       int64  
 2   elapsed_time                int64  
 3   event_name                  object 
 4   name                        object 
 5   level                       int64  
 6   page                        float64
 7   room_coor_x                 float64
 8   room_coor_y                 float64
 9   screen_coor_x               float64
 10  screen_coor_y               float64
 11  hover_duration              float64
 12  text                        object 
 13  fqid                        object 
 14  room_fqid                   object 
 15  text_fqid                   object 
 16  fullscreen                  int64  
 17  hq                          int64  
 18  music                       int64  
 19  level_group   

In [30]:
combined_set['room_position_info']


21702656     931.306091
21702657     931.306091
21702658     965.306091
21702659    1069.306091
21702660    1201.306091
               ...     
7529132             NaN
7529133      337.669530
7529134      526.805622
7529135      534.022889
7529136             NaN
Name: room_position_info, Length: 2605503, dtype: float64

In [28]:
combined_set.groupby(['session_id', 'label']).agg({
    
})

ValueError: No objects to concatenate

In [26]:
import gc
gc.collect()

42